In [1]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import shapiro
from statsmodels.formula.api import ols


In [2]:
CURRENT_DIR = Path.cwd()

In [3]:
filepath = os.path.join(CURRENT_DIR, "S04/S04_residual_stress_imputed_20251222_105458.xlsx")
dfm = pd.read_excel(filepath).rename(
    columns={"Sigma(x)": "sigma_x_pre", "FWHM": "FWHM_pre"}
)
dfm

,idx_excel_post,section,sample_no,location,R,W,D,sigma_x_post,FWHM_post,idx_excel_pre,sigma_x_pre,FWHM_pre,diff_sigma_x
0,4,AA5052,1,1,1400,60,10,13.0,2.55,2,-15.0,2.50,28
1,25,AA5052,2,1,1400,60,15,16.0,2.51,9,2.0,2.47,14
2,46,AA5052,3,1,1400,60,20,19.0,2.47,16,9.0,2.48,10
3,67,AA5052,4,1,1400,70,10,20.0,2.45,23,10.0,2.48,10
4,88,AA5052,5,1,1400,70,15,6.0,2.47,30,0.0,2.49,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,1050,Center,50,7,1600,70,15,2.0,2.45,0,0.0,0.00,2
1130,1071,Center,51,7,1600,70,20,2.0,2.45,0,0.0,0.00,2
1131,1092,Center,52,7,1600,80,10,5.0,2.54,0,0.0,0.00,5
1132,1113,Center,53,7,1600,80,15,1.0,2.41,0,0.0,0.00,1


In [4]:
# Standardize columnes R, W, D

for col in ["R", "W", "D"]:
    mean = dfm[col].mean()
    std = dfm[col].std()
    dfm[col] = (dfm[col] - mean) / std
dfm

,idx_excel_post,section,sample_no,location,R,W,D,sigma_x_post,FWHM_post,idx_excel_pre,sigma_x_pre,FWHM_pre,diff_sigma_x
0,4,AA5052,1,1,-1.224205,-1.224205,-1.224205,13.0,2.55,2,-15.0,2.50,28
1,25,AA5052,2,1,-1.224205,-1.224205,0.000000,16.0,2.51,9,2.0,2.47,14
2,46,AA5052,3,1,-1.224205,-1.224205,1.224205,19.0,2.47,16,9.0,2.48,10
3,67,AA5052,4,1,-1.224205,0.000000,-1.224205,20.0,2.45,23,10.0,2.48,10
4,88,AA5052,5,1,-1.224205,0.000000,0.000000,6.0,2.47,30,0.0,2.49,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,1050,Center,50,7,1.224205,0.000000,0.000000,2.0,2.45,0,0.0,0.00,2
1130,1071,Center,51,7,1.224205,0.000000,1.224205,2.0,2.45,0,0.0,0.00,2
1131,1092,Center,52,7,1.224205,1.224205,-1.224205,5.0,2.54,0,0.0,0.00,5
1132,1113,Center,53,7,1.224205,1.224205,0.000000,1.0,2.41,0,0.0,0.00,1


## Understanding the ANOVA table columns:

- `df`: Degrees of freedom - the number of independent pieces of information for each source
- `sum_sq`: Sum of squares - the total variation attributed to each factor
- `mean_sq`: Mean square - sum of squares divided by degrees of freedom (variance estimate)
- `F`: F-statistic - the ratio of factor variance to residual variance
- `PR(>F)`: P-value - probability of seeing this F-statistic by chance alone

## What is Type

- typ=1 (Type I): sequential SS, each term tested in the order it appears in the formula.
- typ=2, each term (factor or covariate) is tested after all other main effects but ignoring higher‑order interactions that include it.
  This is usually recommended when the model is reasonably balanced and you want tests that respect the marginality principle (main effects evaluated in the presence of other main effects, but not “penalized” by interactions).
- typ=3 (Type III): each term tested after all other terms including interactions; often used in software like SPSS, especially for unbalanced designs, but interpretation of main effects with strong interactions can be tricky.


In [5]:
# Filter the DataFrame for specific sections
filt = (dfm["section"].isin(["AA5052", "AA6061", "Center"])) 
dfa = dfm[filt]

# Define the formula for the model
formula = "diff_sigma_x ~ R + W + D + section"

# Fit the model
model = ols(formula, data=dfa).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model)

# Print the ANOVA results
anova_results.sort_values(by="PR(>F)")

,df,sum_sq,mean_sq,F,PR(>F)
section,2.0,3.369062e+05,168453.080247,136.577818,7.656232e-54
W,1.0,5.733762e+03,5733.761905,4.648800,3.128616e-02
R,1.0,1.898418e+03,1898.417989,1.539193,2.149960e-01
D,1.0,8.786045e+02,878.604497,0.712352,3.988431e-01
Residual,1128.0,1.391259e+06,1233.385356,NaN,NaN


In [6]:
# Filter the DataFrame for specific sections
filt = dfm["section"].isin(["AA5052"]) 
# filt = dfm["section"].isin(["AA5052"]) & (dfm["location"] == 1)
dfa = dfm[filt]

# Define the formula for the model
base = "diff_sigma_x ~ "
t_main = ["R", "W", "D"]
t_interact_2 = ["R:W", "R:D", "W:D"]
t_interact_3 = ["R:W:D"]
t_single = ["I(R)", "I(W)", "I(D)"]
t_quad = ["I(R**2)", "I(W**2)", "I(D**2)"]
t_quad_pair = ["I(R*W)", "I(R*D)", "I(W*D)"]
t_triple_1 = [
    "I(R**2*W)",
    "I(R**2*D)",
    "I(W**2*R)",
    "I(W**2*D)",
    "I(D**2*R)",
    "I(D**2*W)",
]
t_triple_all = ["I(R*W*D)"]

formula = base + " + ".join(
    t_main
    + t_interact_2
    + t_interact_3
    + t_single
    + t_quad
    + t_quad_pair
    + t_triple_1
    + t_triple_all
)
print(formula)

# Fit the model
model = ols(formula, data=dfa).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model)

# Print the ANOVA results
anova_results.sort_values(by="PR(>F)")
display(anova_results)

coefficients = model.params
print(coefficients)

diff_sigma_x ~ R + W + D + R:W + R:D + W:D + R:W:D + I(R) + I(W) + I(D) + I(R**2) + I(W**2) + I(D**2) + I(R*W) + I(R*D) + I(W*D) + I(R**2*W) + I(R**2*D) + I(W**2*R) + I(W**2*D) + I(D**2*R) + I(D**2*W) + I(R*W*D)


,df,sum_sq,mean_sq,F,PR(>F)
R,1.0,3380.670635,3380.670635,12.720647,0.000410
W,1.0,11.146825,11.146825,0.041943,0.837844
D,1.0,667.063492,667.063492,2.509999,0.114002
R:W,1.0,832.595238,832.595238,3.132855,0.077573
R:D,1.0,224.023810,224.023810,0.842947,0.359168
W:D,1.0,1281.523810,1281.523810,4.822065,0.028733
R:W:D,1.0,6.508929,6.508929,0.024492,0.875728
I(R),1.0,25.447592,25.447592,0.095753,0.757165
I(W),1.0,66.500579,66.500579,0.250226,0.617221
I(D),1.0,142.945768,142.945768,0.537870,0.463793


Intercept        21.674603
R                 1.664508
W                 0.069692
D                 0.742302
R:W              -0.742718
R:D              -0.385260
W:D              -0.921447
R:W:D            -0.065698
I(R)              1.664508
I(W)              0.069692
I(D)              0.742302
I(R ** 2)        -1.008826
I(W ** 2)         1.231244
I(D ** 2)         2.621359
I(R * W)         -0.742718
I(R * D)         -0.385260
I(W * D)         -0.921447
I(R ** 2 * W)     0.279015
I(R ** 2 * D)    -2.264561
I(W ** 2 * R)    -1.044683
I(W ** 2 * D)    -0.551541
I(D ** 2 * R)     0.707270
I(D ** 2 * W)    -0.246571
I(R * W * D)     -0.065698
dtype: float64


In [7]:
# Filter the DataFrame for specific sections
filt = dfm["section"].isin(["AA6061"])
dfa = dfm[filt]

# Define the formula for the model
base = "diff_sigma_x ~ "
t_main = ["R", "W", "D"]
t_interact_2 = ["R:W", "R:D", "W:D"]
t_interact_3 = ["R:W:D"]
t_quad = ["I(R**2)", "I(W**2)", "I(D**2)"]
t_quad_pair = ["I(R*W)", "I(R*D)", "I(W*D)"]
t_triple_1 = [
    "I(R**2*W)",
    "I(R**2*D)",
    "I(W**2*R)",
    "I(W**2*D)",
    "I(D**2*R)",
    "I(D**2*W)",
]
t_triple_all = ["I(R*W*D)"]

formula = base + " + ".join(
    t_main
    + t_interact_2
    + t_interact_3
    + t_quad
    + t_quad_pair
    + t_triple_1
    + t_triple_all
)
print(formula)

# Fit the model
model = ols(formula, data=dfa).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model)

# Print the ANOVA results
anova_results.sort_values(by="PR(>F)")

diff_sigma_x ~ R + W + D + R:W + R:D + W:D + R:W:D + I(R**2) + I(W**2) + I(D**2) + I(R*W) + I(R*D) + I(W*D) + I(R**2*W) + I(R**2*D) + I(W**2*R) + I(W**2*D) + I(D**2*R) + I(D**2*W) + I(R*W*D)


,df,sum_sq,mean_sq,F,PR(>F)
R:W:D,1.0,1.320229e+04,13202.285714,3.957055,0.047430
W,1.0,1.211115e+04,12111.146825,3.630013,0.057541
I(R ** 2 * W),1.0,9.587597e+03,9587.597256,2.873642,0.090903
I(D ** 2 * R),1.0,6.536745e+03,6536.744749,1.959226,0.162455
W:D,1.0,6.388667e+03,6388.666667,1.914843,0.167281
I(D ** 2),1.0,4.690065e+03,4690.064815,1.405730,0.236546
D,1.0,4.568766e+03,4568.765873,1.369373,0.242692
I(W ** 2 * R),1.0,4.013617e+03,4013.616746,1.202981,0.273458
I(W ** 2),1.0,2.716255e+03,2716.255291,0.814130,0.367504
R:D,1.0,2.185929e+03,2185.928571,0.655177,0.418801


In [8]:
# Filter the DataFrame for specific sections
filt = dfm["section"].isin(["Center"])
dfa = dfm[filt]

# Define the formula for the model
base = "diff_sigma_x ~ "
t_main = ["R", "W", "D"]
t_interact_2 = ["R:W", "R:D", "W:D"]
t_interact_3 = ["R:W:D"]
t_quad = ["I(R**2)", "I(W**2)", "I(D**2)"]
t_quad_pair = ["I(R*W)", "I(R*D)", "I(W*D)"]
t_triple_1 = [
    "I(R**2*W)",
    "I(R**2*D)",
    "I(W**2*R)",
    "I(W**2*D)",
    "I(D**2*R)",
    "I(D**2*W)",
]
t_triple_all = ["I(R*W*D)"]

formula = base + " + ".join(
    t_main
    + t_interact_2
    + t_interact_3
    + t_quad
    + t_quad_pair
    + t_triple_1
    + t_triple_all
)
print(formula)

# Fit the model
model = ols(formula, data=dfa).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model)

# Print the ANOVA results
anova_results.sort_values(by="PR(>F)")

diff_sigma_x ~ R + W + D + R:W + R:D + W:D + R:W:D + I(R**2) + I(W**2) + I(D**2) + I(R*W) + I(R*D) + I(W*D) + I(R**2*W) + I(R**2*D) + I(W**2*R) + I(W**2*D) + I(D**2*R) + I(D**2*W) + I(R*W*D)


,df,sum_sq,mean_sq,F,PR(>F)
R,1.0,525.777778,525.777778,10.625025,0.001222
W,1.0,315.571429,315.571429,6.377132,0.011987
I(R ** 2 * W),1.0,229.561956,229.561956,4.639035,0.031914
I(R ** 2),1.0,195.047619,195.047619,3.941562,0.047863
R:W:D,1.0,180.035714,180.035714,3.638198,0.057261
D,1.0,91.682540,91.682540,1.852739,0.174314
I(W ** 2 * D),1.0,87.366843,87.366843,1.765527,0.184776
W:D,1.0,81.482143,81.482143,1.646608,0.200244
I(D ** 2),1.0,61.714286,61.714286,1.247135,0.264843
R:W,1.0,52.595238,52.595238,1.062855,0.303255
